In [2]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/all_4h_380.csv")
df.shape

(21717, 13)

In [4]:
import numpy as np
import pandas as pd


def rank_column(df, column, ascending=False):
    _df = df.copy()
    col_name = f"rank_{column}"
    if col_name not in _df.columns:
        _df[col_name] = np.nan
    _df.loc[_df[col_name].isna(), col_name] = _df.groupby("start_time")[column].rank(
        ascending=ascending
    )
    return _df


def remove_volatile_coins(df: pd.DataFrame, n=5):
    """
    remove the most volatile coins, in terms of annualized standard deviation,
    """
    _df = df.copy()
    _df.loc[:, "pct_change"] = _df.groupby(["symbol"])["close"].pct_change() * 100
    rank_volatility = _df.groupby(["symbol"])["pct_change"].std() * np.sqrt(252)
    rank_volatility.sort_values(ascending=False, inplace=True)
    # print removed coins, print left coins
    display(rank_volatility)
    display(rank_volatility.head(n).index)
    _df = _df.loc[~_df["symbol"].isin(rank_volatility.head(n).index)].drop(columns="pct_change")
    return _df

In [5]:
_ = remove_volatile_coins(df, 5)
display(_.shape)
_.head()

symbol
WIF       106.215167
kFLOKI     90.496799
kPEPE      84.003606
kBONK      78.204201
kSHIB      66.841349
ONDO       63.988691
UNI        62.992862
W          62.545467
MAVIA      61.393839
AR         61.147989
JTO        60.999410
GALA       59.932344
JUP        59.629912
FET        59.193988
WLD        58.579627
PENDLE     56.667776
FTM        56.019685
NEAR       52.912136
RNDR       52.459399
DOGE       51.966288
PYTH       51.964551
HBAR       51.371936
BCH        50.519497
NEO        49.881957
CFX        49.881589
RUNE       49.748193
ORDI       49.477361
ENS        47.488574
STRK       47.479554
SEI        47.447563
TON        47.127223
INJ        46.828396
TIA        46.492051
APT        45.779305
STX        45.695714
XAI        44.932472
FIL        44.692940
SUI        43.296619
AVAX       42.300670
COMP       41.874185
OP         41.173938
APE        40.731371
SOL        40.691332
GMX        40.454573
LDO        40.372355
ARB        36.997575
MKR        36.718979
ADA   

Index(['WIF', 'kFLOKI', 'kPEPE', 'kBONK', 'kSHIB'], dtype='object', name='symbol')

(19812, 13)

,symbol,datetime,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
0,BTC,2024-02-22-16:00:00,2024-02-22-16:00:00,2024-02-22-19:59:59,4h,18207,51522.0,51750.0,51404.0,51607.0,735.64027,1708617600000,1708631999999
1,BTC,2024-02-22-20:00:00,2024-02-22-20:00:00,2024-02-22-23:59:59,4h,19355,51292.0,52116.0,51192.0,51512.0,650.88869,1708632000000,1708646399999
2,BTC,2024-02-23-00:00:00,2024-02-23-00:00:00,2024-02-23-03:59:59,4h,16942,51121.0,51550.0,50990.0,51292.0,474.88428,1708646400000,1708660799999
3,BTC,2024-02-23-04:00:00,2024-02-23-04:00:00,2024-02-23-07:59:59,4h,16314,50959.0,51350.0,50915.0,51120.0,423.56810,1708660800000,1708675199999
4,BTC,2024-02-23-08:00:00,2024-02-23-08:00:00,2024-02-23-11:59:59,4h,19465,51261.0,51301.0,50865.0,50959.0,506.30719,1708675200000,1708689599999


In [6]:
def calculate_normalized_area_under_ema200(df):
    _df = df.copy()

    # calculate ema200
    _df["ema200"] = _df.groupby("symbol")["close"].transform(lambda x: x.ewm(span=200, adjust=False).mean())
    _df["area"] = _df["close"] - _df["ema200"]

    # normalize the area base on the mean of the ema200 for the last 180 candles
    # _df["norm_area"] = _df["area"] / _df["ema200"].mean() for signal coin

    # now work on multi coin df
    _df["norm_area"] = _df.groupby("symbol").apply(
        lambda group: group["area"] / group["ema200"].mean()
    ).reset_index(level=0, drop=True)

    _df["sum_norm_area"] = _df.groupby('symbol')['norm_area'].transform(lambda x: x.rolling(window=180).sum())

    return _df

In [7]:
_ = calculate_normalized_area_under_ema200(_)
_

/var/folders/6w/7b7qhtd56jz545z6zwv6zc800000gn/T/ipykernel_49649/3238306682.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  _df["norm_area"] = _df.groupby("symbol").apply(


,symbol,datetime,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc,ema200,area,norm_area,sum_norm_area
0,BTC,2024-02-22-16:00:00,2024-02-22-16:00:00,2024-02-22-19:59:59,4h,18207,51522.000,51750.000,51404.000,51607.000,735.64027,1708617600000,1708631999999,51522.000000,0.000000,0.000000,NaN
1,BTC,2024-02-22-20:00:00,2024-02-22-20:00:00,2024-02-22-23:59:59,4h,19355,51292.000,52116.000,51192.000,51512.000,650.88869,1708632000000,1708646399999,51519.711443,-227.711443,-0.003676,NaN
2,BTC,2024-02-23-00:00:00,2024-02-23-00:00:00,2024-02-23-03:59:59,4h,16942,51121.000,51550.000,50990.000,51292.000,474.88428,1708646400000,1708660799999,51515.744165,-394.744165,-0.006372,NaN
3,BTC,2024-02-23-04:00:00,2024-02-23-04:00:00,2024-02-23-07:59:59,4h,16314,50959.000,51350.000,50915.000,51120.000,423.56810,1708660800000,1708675199999,51510.204422,-551.204422,-0.008897,NaN
4,BTC,2024-02-23-08:00:00,2024-02-23-08:00:00,2024-02-23-11:59:59,4h,19465,51261.000,51301.000,50865.000,50959.000,506.30719,1708675200000,1708689599999,51507.724776,-246.724776,-0.003983,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21712,ENS,2024-04-25-08:00:00,2024-04-25-08:00:00,2024-04-25-11:59:59,4h,2540,14.379,14.538,14.125,14.384,17044.20000,1714032000000,1714046399999,17.579713,-3.200713,-0.148171,-22.899125
21713,ENS,2024-04-25-12:00:00,2024-04-25-12:00:00,2024-04-25-15:59:59,4h,2477,14.905,14.921,14.220,14.377,43899.09000,1714046400000,1714060799999,17.553099,-2.648099,-0.122589,-23.025696
21714,ENS,2024-04-25-16:00:00,2024-04-25-16:00:00,2024-04-25-19:59:59,4h,1999,14.780,14.926,14.680,14.891,9062.52000,1714060800000,1714075199999,17.525506,-2.745506,-0.127098,-23.148396
21715,ENS,2024-04-25-20:00:00,2024-04-25-20:00:00,2024-04-25-23:59:59,4h,1435,14.646,14.869,14.618,14.784,3667.08000,1714075200000,1714089599999,17.496854,-2.850854,-0.131975,-23.287703


In [12]:
def rank_symbols_by_area(df):
    """
    This function takes a DataFrame with columns 'symbol', 'datetime', 'sum_norm_area'
    and returns a DataFrame with each symbol's latest datetime, rank based on sum_norm_area,
    and the sum_norm_area itself, sorted in descending order of sum_norm_area.

    Parameters:
    df (pd.DataFrame): DataFrame containing the symbol, datetime, and sum_norm_area columns.

    Returns:
    pd.DataFrame: DataFrame with columns 'symbol', 'datetime', 'rank', and 'sum_norm_area'.
    """

    # Step 1: Find the latest datetime for each symbol
    latest_dates = df.groupby('symbol')['datetime'].max().reset_index()

    # Step 2: Merge the original DataFrame with the latest_dates to get the latest sum_norm_area
    latest_sums = df.merge(latest_dates, on=['symbol', 'datetime'])

    # Step 3: Sort by sum_norm_area and calculate ranks
    latest_sums_sorted = latest_sums.sort_values(by='sum_norm_area', ascending=False)
    latest_sums_sorted['rank'] = latest_sums_sorted['sum_norm_area'].rank(method='dense', ascending=False)

    # Final DataFrame with required columns
    # final_df = latest_sums_sorted[['symbol', 'datetime', 'rank', 'sum_norm_area']]
    final_df = latest_sums_sorted[['symbol', 'start_time', 'rank', 'end_time']]

    return final_df.reset_index(drop=True)

In [15]:
_df = rank_symbols_by_area(_)
_df.info()
# parse times to pd.datatime

_df['start_time'] = pd.to_datetime(_df['start_time'])
_df['end_time'] = pd.to_datetime(_df['end_time'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   symbol      52 non-null     object 
 1   start_time  52 non-null     object 
 2   rank        52 non-null     float64
 3   end_time    52 non-null     object 
dtypes: float64(1), object(3)
memory usage: 1.8+ KB
